# Measurements of an MPS

This notebook is essentially an edited version of http://itensor.org/docs.cgi?vers=julia&page=formulas/measure_mps

In this notebook, we will be able to take measurements within a matrix product state.


In [2]:
using ITensors
print("Done")

Done

In [11]:
N = 10
bondlength = 2
sites = siteinds("S=1/2", N) # we are creating site indices with S=1/2.
psi = randomMPS(sites, bondlength) # here we are creating a random MPS that we will take measurements of

print(psi[1]) # note that you can print out portions of the MPS

ITensor ord=2
Dim 1: (dim=2|id=114|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=211|"Link,l=1")
NDTensors.Dense{Float64,Array{Float64,1}}
 2×2
 0.3417410312235465   0.9111980937242931
 0.21536512738493463  0.08092566637323806

In [12]:
for j=1:length(psi) # for loop running between all the the sites (size of the MPS)
    
    orthogonalize!(psi,j) # shifts the orthogonality center of thje MPS to site j
    
    s = siteind(psi, j) # retrieves the site index of the jth MPS tesnor. 
    
    val = scalar(psi[j]*op(s, "Sz")*dag(prime(psi[j],s))) # This is the actual calculation
    # We take psi[j] and the dagger of psi[j] and sandwich the operator, Sz in between in this example.
    # s carries the tag of the site. Note that we have set our sites to be S = 1/2. 
     
    println("$j $val") # using $variable_name will print out the value of the variable
end
    

1 0.4470688984285184
2 -0.2491167699875433
3 -0.024733301166914787
4 -0.1646014275283429
5 0.00570543784977022
6 -0.31473057435968993
7 0.22788254698734173
8 -0.3141772986009457
9 -0.3305831695620387
10 0.1901927018702796


To get a better understanding of hwo this works, let's consider an ising model example as we did in the previous notebook.

In [27]:
N = 10 # number of sites. To be simple, let's default to N=10
Jzz = 1.0 # Nearest-Neighbor interaction strength
Jx = 0.0 # transverse field interaction strength

sites = siteinds("S=1/2", N)

ampo = AutoMPO()
for j=1:N-1 
    ampo += Jx, "Sx", j
    ampo += Jzz, "Sz", j, "Sz", j+1 #two body operator
end
ampo += Jx, "Sx", N

H = MPO(ampo, sites) 
sweeps = Sweeps(10)
maxdim!(sweeps, 10, 20, 100, 100, 200)
cutoff!(sweeps, 1E-10)
psi0 = randomMPS(sites, 2) # random MPS of bond dimension 2
energy, psi_ising = dmrg(H, psi0, sweeps)

#note that I remove the let statement in order to return psi_ising.

print(psi_ising)

After sweep 1 energy=-2.249998381627 maxlinkdim=6 time=0.039
After sweep 2 energy=-2.250000000000 maxlinkdim=1 time=0.025
After sweep 3 energy=-2.250000000000 maxlinkdim=1 time=0.015
After sweep 4 energy=-2.250000000000 maxlinkdim=1 time=0.014
After sweep 5 energy=-2.250000000000 maxlinkdim=1 time=0.010
After sweep 6 energy=-2.250000000000 maxlinkdim=1 time=0.011
After sweep 7 energy=-2.250000000000 maxlinkdim=1 time=0.017
After sweep 8 energy=-2.250000000000 maxlinkdim=1 time=0.020
After sweep 9 energy=-2.250000000000 maxlinkdim=1 time=0.017
After sweep 10 energy=-2.250000000000 maxlinkdim=1 time=0.012
MPS
[1] IndexSet{2} (dim=1|id=8|"Link,l=1") (dim=2|id=75|"S=1/2,Site,n=1") 
[2] IndexSet{3} (dim=1|id=362|"Link,l=2") (dim=2|id=526|"S=1/2,Site,n=2") (dim=1|id=8|"Link,l=1") 
[3] IndexSet{3} (dim=1|id=385|"Link,l=3") (dim=2|id=697|"S=1/2,Site,n=3") (dim=1|id=362|"Link,l=2") 
[4] IndexSet{3} (dim=1|id=784|"Link,l=4") (dim=2|id=564|"S=1/2,Site,n=4") (dim=1|id=385|"Link,l=3") 
[5] IndexSet

In [28]:
psi_ising

MPS
[1] IndexSet{2} (dim=1|id=8|"Link,l=1") (dim=2|id=75|"S=1/2,Site,n=1") 
[2] IndexSet{3} (dim=1|id=362|"Link,l=2") (dim=2|id=526|"S=1/2,Site,n=2") (dim=1|id=8|"Link,l=1") 
[3] IndexSet{3} (dim=1|id=385|"Link,l=3") (dim=2|id=697|"S=1/2,Site,n=3") (dim=1|id=362|"Link,l=2") 
[4] IndexSet{3} (dim=1|id=784|"Link,l=4") (dim=2|id=564|"S=1/2,Site,n=4") (dim=1|id=385|"Link,l=3") 
[5] IndexSet{3} (dim=1|id=423|"Link,l=5") (dim=2|id=667|"S=1/2,Site,n=5") (dim=1|id=784|"Link,l=4") 
[6] IndexSet{3} (dim=1|id=345|"Link,l=6") (dim=2|id=628|"S=1/2,Site,n=6") (dim=1|id=423|"Link,l=5") 
[7] IndexSet{3} (dim=1|id=559|"Link,l=7") (dim=2|id=669|"S=1/2,Site,n=7") (dim=1|id=345|"Link,l=6") 
[8] IndexSet{3} (dim=1|id=598|"Link,l=8") (dim=2|id=284|"S=1/2,Site,n=8") (dim=1|id=559|"Link,l=7") 
[9] IndexSet{3} (dim=1|id=715|"Link,l=9") (dim=2|id=332|"S=1/2,Site,n=9") (dim=1|id=598|"Link,l=8") 
[10] IndexSet{2} (dim=2|id=877|"S=1/2,Site,n=10") (dim=1|id=715|"Link,l=9") 


In [29]:
# We now have a psi from the last sweep above.

for j=1:length(psi_ising) # for loop running between all the the sites (size of the MPS)
    
    orthogonalize!(psi_ising,j) # shifts the orthogonality center of thje MPS to site j
    
    s = siteind(psi_ising, j) # retrieves the site index of the jth MPS tesnor. 
    
    val = scalar(psi_ising[j]*op(s, "Sz")*dag(prime(psi_ising[j],s))) # This is the actual calculation
    # We take psi[j] and the dagger of psi[j] and sandwich the operator, Sz in between in this example.
    # s carries the tag of the site. Note that we have set our sites to be S = 1/2. 
     
    println("$j $val") # using $variable_name will print out the value of the variable
end

1 0.5
2 -0.5
3 0.5
4 -0.5
5 0.5
6 -0.5
7 0.5
8 -0.5
9 0.5
10 -0.5


Using $Jzz = 1.0$ and $Jx = 0.0$, we expect the spins to be completely antialigned. This means, the measurement of this operator should always be plus or minus 1/2, but should alternate!